In [1]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.utils.testing import all_estimators

from sklearn.linear_model import RidgeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import MultinomialNB, BernoulliNB, CategoricalNB, ComplementNB
from sklearn.linear_model import LogisticRegression, SGDClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
import joblib
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import accuracy_score
from statistics import mean

def get_scores(X):
    conf_scores = vote_model.transform(X)

    scores = []
    for s in conf_scores:
        all_classes = []
        for i in range(int(len(s)/2)):
            all_classes.append(s[2*i+1])
        score = mean(all_classes)*4-2
        if score < 0:
            if score > -0.1:
                scores.append(['left', 0])
            elif score > -0.8:
                scores.append(['left', 0.3])
            elif score > -1.1:
                scores.append(['left', 0.6])
            else:
                scores.append(['left', 1])
        else:
            if score < 0.1:
                scores.append(['right', 0])
            elif score < 0.8:
                scores.append(['right', 0.3])
            elif score > -0.8:
                scores.append(['right', 0.6])
            else:
                scores.append(['right', 1])
    return scores

def clean(x):
    
    stemmer = SnowballStemmer('english')
    words = stopwords.words("english")

    l = []
    for i in re.sub("[^a-zA-Z]", " ", x).split():
        if i not in words:
            l.append(stemmer.stem(i))
    
    return " ".join(l).lower()

# def features():
    

articles = pd.read_csv('binary_bias_extreme.csv')

#randomizes rows
articles = articles.sample(frac=1)

#removes non-letters, stopwords, and stems words
articles['cleaned'] = articles['article'].apply(clean)

X_train, X_test, y_train, y_test = train_test_split(articles['cleaned'], articles['bias'], test_size=0.2)

### CLASSIFIER ###

estimator = []
estimator.append(('MNB', MultinomialNB()))
estimator.append(('BNB', BernoulliNB()))
estimator.append(('LR', LogisticRegression(max_iter=10000)))
estimator.append(('SGD', SVC(probability=True)))

pipeline = Pipeline([('vect', CountVectorizer(stop_words='english', ngram_range=(2, 3))),
                     ('best',  SelectKBest(k=10000)),
                     ('model', VotingClassifier(estimators = estimator, voting ='soft', flatten_transform=True))])

vote_model = pipeline.fit(X_train, y_train)
joblib.dump(vote_model, 'vote_model.joblib')

print("Accuracy: " + str(round(vote_model.score(X_test, y_test)*100, 2)) + "%")

scores = get_scores(X_test)

binary = []
for s in scores:
    if s[0] == 'left':
        binary.append(0)
    else:
        binary.append(1)

print(f'Adjusted: {accuracy_score(y_test, binary)}%')

D:\Anaconda\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


Accuracy: 69.19%
Adjusted: 0.6919191919191919%
